In [1]:
pip install pdfplumber sumy transformers nltk

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   - -----------------------

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import nltk
nltk.download("punkt")

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
import pdfplumber
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from transformers import pipeline
import os

In [3]:
def load_text(file_path):
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError("Unsupported file format. Use .pdf or .txt")
    return text.strip()

In [4]:
def extractive_summary(text,num_sentences=5):
    parser=PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer=TextRankSummarizer()
    summary=summarizer(parser.document, num_sentences)
    return "\n".join(str(sentence) for sentence in summary)

In [5]:
def abstractive_summary(text, model_name="sshleifer/distilbart-cnn-12-6"):
    summarizer=pipeline("summarization",model=model_name)
    max_chunk=1024
    text=text.replace("\n", " ")
    chunks=[text[i:i+max_chunk] for i in range(0,len(text),max_chunk)]
    
    summary=""
    for chunk in chunks:
        result=summarizer(chunk,max_length=150,min_length=40, do_sample=False)
        summary+=result[0]['summary_text'] + " "
    return summary.strip()

In [6]:
def summarize_document(file_path,output_format="text",abstractive=True,extractive=True):
    text=load_text(file_path)
    
    results={}
    if extractive:
        results["Extractive Summary"]=extractive_summary(text)
    if abstractive:
        results["Abstractive Summary"]=abstractive_summary(text)
    
    if output_format=="markdown":
        output="\n\n".join(f"### {k}\n\n{v}" for k, v in results.items())
    else:
        output="\n\n".join(f"{k}:\n{v}" for k, v in results.items())
    
    return output

In [7]:
file_path="C:\\Users\\Akshata Pandit\\Downloads\\Document2.pdf"
summary_output=summarize_document(file_path)
print(summary_output)

Device set to use cpu


Extractive Summary:
Country of Origin: UNITED KINGDOM Commodity Code 90318080 Export CPC 10 00 001 Company Details: Registered No: 1418812 England Registered Office: New Mills, Wotton-under-Edge Gloucestershire GL12 8JR U.K. VAT No: GB 422-9005-81 EORI No: GB 422-9005-81-005 Continued… Id: 436829 GBWLCommercial.rpt v1.19 Page 2 of 8 Renishaw UK Sales Limited Tel: +44 (0) 1453 524524 New Mills Fax: +44 (0) 1453 524901 Wotton-under-Edge E-mail: sales.office@renishaw.com Gloucestershire GL12 8JR Web: www.renishaw.com United Kingdom EORI No: GB 422-9005-81-005 Dated: 05 Jan 2024 Time: 08:19 Commercial Invoice Invoice No: S2401-34 Carton Order No Part Number Description Unit Price Qty Sales Value Customs Value
Country of Origin: UNITED KINGDOM Commodity Code 90319000 Export CPC 10 00 001 Company Details: Registered No: 1418812 England Registered Office: New Mills, Wotton-under-Edge Gloucestershire GL12 8JR U.K. VAT No: GB 422-9005-81 EORI No: GB 422-9005-81-005 Continued… Id: 436829 GBWLCom

In [16]:
file_path="C:\\Users\\Akshata Pandit\\Downloads\\Document1.pdf"
summary_output=summarize_document(file_path)
print(summary_output)

Device set to use cpu
Error while downloading from https://cdn-lfs.hf.co/sshleifer/distilbart-cnn-12-6/1e46814333b97dfa0f866f58fd15cd7b48ffbe7fd4c1a929caa5f95c7b2fa592?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747588649&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzU4ODY0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zc2hsZWlmZXIvZGlzdGlsYmFydC1jbm4tMTItNi8xZTQ2ODE0MzMzYjk3ZGZhMGY4NjZmNThmZDE1Y2Q3YjQ4ZmZiZTdmZDRjMWE5MjljYWE1Zjk1YzdiMmZhNTkyP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=P1BjnrLlFPe6HOoMAx%7EZWeRYXotRMk3gPqCOhR5BeFJyMz6gL8XQOfKqZVJyJfTfJHtK8ZOccO5E0k7KeOBY3qyOYnm%7EH5JkszqNAcllnisG7e9WxCQg4Lsgyb%7EReKh5YOJPjiIKt5cTaINo73Wj%7Ee7hh9qD1BZny7FyzM%7EcyTQiI735697nUYtw1P%7EQTee%7EwHRgG3YnJaFXI-EZQy6D99cUM3%7EMVEEn6ngauo6Mtn7-kfLngVxkUdvw-Bolyd8pJnwmQoExU9gqQiqBSJoipthqSJqnBLp212xKVSpFZsaGpyc44XZn10X1o0aGuxTE3%7E2Z%7EcKip5vYknMiLk

model.safetensors:  25%|##4       | 398M/1.62G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/sshleifer/distilbart-cnn-12-6/1e46814333b97dfa0f866f58fd15cd7b48ffbe7fd4c1a929caa5f95c7b2fa592?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747588649&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzU4ODY0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zc2hsZWlmZXIvZGlzdGlsYmFydC1jbm4tMTItNi8xZTQ2ODE0MzMzYjk3ZGZhMGY4NjZmNThmZDE1Y2Q3YjQ4ZmZiZTdmZDRjMWE5MjljYWE1Zjk1YzdiMmZhNTkyP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=P1BjnrLlFPe6HOoMAx%7EZWeRYXotRMk3gPqCOhR5BeFJyMz6gL8XQOfKqZVJyJfTfJHtK8ZOccO5E0k7KeOBY3qyOYnm%7EH5JkszqNAcllnisG7e9WxCQg4Lsgyb%7EReKh5YOJPjiIKt5cTaINo73Wj%7Ee7hh9qD1BZny7FyzM%7EcyTQiI735697nUYtw1P%7EQTee%7EwHRgG3YnJaFXI-EZQy6D99cUM3%7EMVEEn6ngauo6Mtn7-kfLngVxkUdvw-Bolyd8pJnwmQoExU9gqQiqBSJoipthqSJqnBLp212xKVSpFZsaGpyc44XZn10X1o0aGuxTE3%7E2Z%7EcKip5vYknMiLkAeww__&Key-Pair-Id=K3R

model.safetensors:  26%|##6       | 440M/1.66G [00:00<?, ?B/s]

Extractive Summary:
Country of Origin: UNITED KINGDOM Commodity Code 90318080 Export CPC 10 00 001 Company Details: Registered No: 1418812 England Registered Office: New Mills, Wotton-under-Edge Gloucestershire GL12 8JR U.K. VAT No: GB 422-9005-81 EORI No: GB 422-9005-81-005 Continued… Id: 436829 GBWLCommercial.rpt v1.19 Page 2 of 8 Renishaw UK Sales Limited Tel: +44 (0) 1453 524524 New Mills Fax: +44 (0) 1453 524901 Wotton-under-Edge E-mail: sales.office@renishaw.com Gloucestershire GL12 8JR Web: www.renishaw.com United Kingdom EORI No: GB 422-9005-81-005 Dated: 05 Jan 2024 Time: 08:19 Commercial Invoice Invoice No: S2401-34 Carton Order No Part Number Description Unit Price Qty Sales Value Customs Value
Country of Origin: UNITED KINGDOM Commodity Code 90319000 Export CPC 10 00 001 Company Details: Registered No: 1418812 England Registered Office: New Mills, Wotton-under-Edge Gloucestershire GL12 8JR U.K. VAT No: GB 422-9005-81 EORI No: GB 422-9005-81-005 Continued… Id: 436829 GBWLCom

In [18]:
file_path="C:\\Users\\Akshata Pandit\\Downloads\\Document5.pdf"
summary_output=summarize_document(file_path)
print(summary_output)

Device set to use cpu
Your max_length is set to 150, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


Extractive Summary:
CAsh fLoW stAteMent for the YeAr ended MArCh 31, 2023 (Amount ` in lakhs) Year ended March Year ended March 31, 2023 31, 2022 (A) CAsh fLoW froM operAtIng ACtIvItIes profIt before tAX 2,362.19 1,203.98 Adjustments for : Depreciation and Amortization 4.97 7.83 Provision for Employee Benefits (Net of reversal) (15.01) (105.30) Provision for Expected Credit Loss for trade receivables 331.73 (25.46) FVTPL (Gain)/Loss of current investments (145.46) (12.15) FVTPL (Gain)/Loss of non-current investments (301.24) (202.02) Provision for diminution in value of investment - 15.22 (Profit)/Loss on sale of non-current investments - 243.50 (Profit)/Loss on sale of current investments (54.46) (121.71) (Profit)/Loss on sale of Property, Plant and Equipment (Net) (7.33) (0.29) Interest Income (9.45) (82.19) Dividend Income from current investments 18 (2,604.80) (1,507.72) Operating Profit before working capital changes (438.86) (586.31) Changes in working capital : Adjustments for (

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]